<a href="https://colab.research.google.com/github/bidulgiya999/medgemma_test/blob/main/medgemma_4b_it.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U transformers

In [ ]:
from huggingface_hub import login
login(new_session=False)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# # pip install accelerate
# from transformers import AutoProcessor, AutoModelForImageTextToText
# from PIL import Image
# import requests
# import torch

# model_id = "google/medgemma-4b-it"

# model = AutoModelForImageTextToText.from_pretrained(
#     model_id,
#     torch_dtype=torch.bfloat16,
#     device_map="auto",
# )
# processor = AutoProcessor.from_pretrained(model_id)

# # Image attribution: Stillwaterising, CC0, via Wikimedia Commons
# # image_dir = "https://vanguardmedical.ca/wp-content/uploads/2024/03/A-Comprehensive-Guide-to-Treating-Forehead-Wrinkles.jpg"
# # image = Image.open(requests.get(image_dir, headers={"User-Agent": "example"}, stream=True).raw)

# from PIL import Image

# # 2. image_path 변수에 로컬 이미지 파일의 경로를 지정합니다.
# image_path = "/content/drive/MyDrive/medgemma/frontfrace.jpg"

# # 3. Image.open() 함수를 사용하여 로컬 이미지를 불러옵니다.
# image = Image.open(image_path)

# messages = [
#     {
#         "role": "system",
#         "content": [{"type": "text", "text": "Please evaluate this person's forehead wrinkles, forehead pigmentation, glabellar lines (between eyebrows), crow's feet (eye wrinkles), cheek pigmentation, and cheek pores separately. Also, rate the severity of each on a scale of 0-6.."}]
#     },
#     {
#         "role": "user",
#         "content": [
#             {"type": "text", "text": "Could you provide me with the clinical findings of the skin status? "},
#             {"type": "image", "image": image}
#         ]
#     }
# ]

# inputs = processor.apply_chat_template(
#     messages, add_generation_prompt=True, tokenize=True,
#     return_dict=True, return_tensors="pt"
# ).to(model.device, dtype=torch.bfloat16)

# input_len = inputs["input_ids"].shape[-1]

# with torch.inference_mode():
#     generation = model.generate(**inputs, max_new_tokens=2000, do_sample=False)
#     generation = generation[0][input_len:]

# decoded = processor.decode(generation, skip_special_tokens=True)
# print(decoded)
# 이 코드를 수정하고싶어 parameter에서 temperature를 넣고싶어 어떻게 수정을 해야할까?

#                     "Please evaluate this person according to the following criteria and classify the severity for each step",
#                     "Forehead hyperpigmentation: 0-5 scale",
#                     "Forehead wrinkles: 0-6 scale",
#                     "Glabellar wrinkles (between eyebrows): 0-6 scale",
#                     "Left cheek pores: 0-5 scale",
#                     "Left cheek hyperpigmentation: 0-5 scale",
#                     "Right cheek pores: 0-5 scale",
#                     "Right cheek hyperpigmentation: 0-5 scale",
#                     "Lip dryness: 0-4 scale"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# pip install accelerate
from transformers import AutoProcessor, AutoModelForImageTextToText
from PIL import Image
import torch

model_id = "google/medgemma-4b-it"

model = AutoModelForImageTextToText.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
processor = AutoProcessor.from_pretrained(model_id)

# 이미지 로드
image_path = "/content/drive/MyDrive/medgemma/test_face.jpg"
image = Image.open(image_path).convert("RGB")

# ✅ system 텍스트를 하나의 문자열로 합치기
criteria_lines = [
    "Please evaluate this person according to the following criteria and classify the severity for each step:",
    "Forehead hyperpigmentation: 0-5 scale",
    "Forehead wrinkles: 0-6 scale",
    "Glabellar wrinkles (between eyebrows): 0-6 scale",
    "Left cheek pores: 0-5 scale",
    "Left cheek hyperpigmentation: 0-5 scale",
    "Right cheek pores: 0-5 scale",
    "Right cheek hyperpigmentation: 0-5 scale",
    "Lip dryness: 0-4 scale",
]
system_text = "\n".join(criteria_lines)  # ← 단일 문자열

messages = [
    {
        "role": "system",
        "content": [
            {"type": "text", "text": system_text}
        ],
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "Could you provide me with the clinical findings of the skin status? "},
            {"type": "image"}  # 이미지 토큰만
        ],
    },
]

# 1) 문자열 프롬프트 생성
chat_prompt = processor.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=False  # 문자열로 받기
)

# 2) 텍스트+이미지 함께 인코딩
inputs = processor(
    text=chat_prompt,
    images=[image],
    return_tensors="pt"
).to(model.device)

input_len = inputs["input_ids"].shape[-1]

# 3) 생성 (샘플링/temperature 적용)
with torch.inference_mode():
    generation = model.generate(
        **inputs,
        max_new_tokens=2000,
        do_sample=True,
        temperature=0.3,
        top_p=0.9,
        top_k=50
    )
    generation = generation[0][input_len:]

decoded = processor.decode(generation, skip_special_tokens=True)
print(decoded)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Okay, I will evaluate the skin based on the provided criteria and scale.

**Clinical Findings:**

*   **Forehead hyperpigmentation:** 2
*   **Forehead wrinkles:** 3
*   **Glabellar wrinkles (between eyebrows):** 4
*   **Left cheek pores:** 2
*   **Left cheek hyperpigmentation:** 2
*   **Right cheek pores:** 2
*   **Right cheek hyperpigmentation:** 2
*   **Lip dryness:** 3

**Explanation of Ratings:**

*   **Forehead hyperpigmentation:** There is some mild hyperpigmentation (darkening) on the forehead.
*   **Forehead wrinkles:** There are some mild wrinkles on the forehead.
*   **Glabellar wrinkles (between eyebrows):** There are some mild to moderate wrinkles between the eyebrows.
*   **Left cheek pores:** There are some mild pores on the left cheek.
*   **Left cheek hyperpigmentation:** There is some mild hyperpigmentation on the left cheek.
*   **Right cheek pores:** There are some mild pores on the right cheek.
*   **Right cheek hyperpigmentation:** There is some mild hyperpigmentat

In [ ]:
# pip install accelerate
from transformers import AutoProcessor, AutoModelForImageTextToText
from PIL import Image
import torch

model_id = "google/medgemma-4b-it"

model = AutoModelForImageTextToText.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
processor = AutoProcessor.from_pretrained(model_id)

# 이미지 로드
image_path = "/content/drive/MyDrive/medgemma/test_face.jpg"
image = Image.open(image_path).convert("RGB")

# ✅ system 텍스트를 하나의 문자열로 합치기
criteria_lines = [
    "Please evaluate this person according to the following criteria and classify the severity for each step:",
    "Forehead hyperpigmentation: 0-5 scale",
    "Forehead wrinkles: 0-6 scale",
    "Glabellar wrinkles (between eyebrows): 0-6 scale",
    "Left cheek pores: 0-5 scale",
    "Left cheek hyperpigmentation: 0-5 scale",
    "Right cheek pores: 0-5 scale",
    "Right cheek hyperpigmentation: 0-5 scale",
    "Lip dryness: 0-4 scale",
]
system_text = "\n".join(criteria_lines)  # ← 단일 문자열

messages = [
    {
        "role": "system",
        "content": [
            {"type": "text", "text": system_text}
        ],
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "Could you provide me with the clinical findings of the skin status? "},
            {"type": "image"}  # 이미지 토큰만
        ],
    },
]

# 1) 문자열 프롬프트 생성
chat_prompt = processor.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=False  # 문자열로 받기
)

# 2) 텍스트+이미지 함께 인코딩
inputs = processor(
    text=chat_prompt,
    images=[image],
    return_tensors="pt"
).to(model.device)

input_len = inputs["input_ids"].shape[-1]

# 3) 생성 (샘플링/temperature 적용)
with torch.inference_mode():
    generation = model.generate(
        **inputs,
        max_new_tokens=2000,
        do_sample=True,
        temperature=0.5,
        top_p=0.9,
        top_k=50
    )
    generation = generation[0][input_len:]

decoded = processor.decode(generation, skip_special_tokens=True)
print(decoded)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Okay, I will evaluate the provided image based on the criteria you provided and give a severity rating for each.

**Clinical Findings:**

*   **Forehead hyperpigmentation:** 2
*   **Forehead wrinkles:** 3
*   **Glabellar wrinkles (between eyebrows):** 4
*   **Left cheek pores:** 2
*   **Left cheek hyperpigmentation:** 2
*   **Right cheek pores:** 2
*   **Right cheek hyperpigmentation:** 2
*   **Lip dryness:** 3

**Explanation of Ratings:**

*   **Forehead hyperpigmentation:** The forehead shows some areas of increased pigmentation, likely sunspots or age spots.
*   **Forehead wrinkles:** There are some wrinkles present on the forehead, indicating some degree of aging.
*   **Glabellar wrinkles (between eyebrows):** The glabella area has noticeable wrinkles, which are common with age.
*   **Left cheek pores:** The left cheek has some visible pores, which are normal.
*   **Left cheek hyperpigmentation:** There is some hyperpigmentation on the left cheek, possibly related to sun exposure o

In [ ]:
##### 여기선 파인튜닝 기법 적용

In [ ]:
!pip install -q -U transformers bitsandbytes accelerate peft datasets pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.4/503.4 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 52.9 MB/s eta 0:00:00
ERROR: Operation cancelled by user
